<a href="https://colab.research.google.com/github/kabilan942/Legal/blob/main/LDS_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install transformers==4.21.0
!pip install sentencepiece
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM 
import nltk
!pip install rouge
nltk.download('punkt')
from rouge import Rouge
!pip install datasets
from datasets import Dataset, DatasetDict
!pip install rouge_score
!pip install datasets evaluate transformers[sentencepiece]
!pip install nltk
from nltk.tokenize import sent_tokenize
import pandas as pd
import numpy as np
import glob
import nltk
nltk.download('punkt')
from nltk import tokenize
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
import torch.nn.functional as F
import torch
import json
import random
import os
import sys
sys.path.insert(0, '../')
from IPython.display import display, HTML
import torch
from datasets import load_dataset, load_metric, Dataset
from transformers import LEDTokenizer, LEDForConditionalGeneration
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import evaluate
rouge_score = evaluate.load('rouge')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


##CLS Chunking

In [5]:
from transformers import BertModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
bert_model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states = False)
bert_model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

**Have to change path**

In [6]:
def get_root_path():
    '''
    function to get root path of dataset
    change the path variable to the path of the dataset
    '''
    path = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/Single/"
    return path

def get_summary_data():
    '''
    function to get names, documents, and summaries
    change the path variable to the path of the dataset
    '''
    
    path = get_root_path() + '/judgement'
    all_files = glob.glob(path + "/*.txt")
    data_source = []
    names = []
    for filename in all_files:
        with open(filename, 'r') as f:
            p = filename.rfind("/")
            names.append(filename[p+1:])
            a = f.read()
            data_source.append(a)
    path = get_root_path() + '/summary'
    all_files = glob.glob(path + "/*.txt")
    data_summary = []
    for filename in all_files:
        with open(filename, 'r') as f: 
            a = f.read()
            l = len(a)
            data_summary.append(a)
            
    return names, data_source, data_summary

def split_to_sentences(para):
    sents = nltk.sent_tokenize(para)   # returns list of sentences from para
    return sents


In [160]:
names, data_source, data_summary = get_summary_data()
print(len(names))
print(len(data_source))
print(len(data_summary))


15
15
15


In [161]:
len(data_source[0]), len(data_summary[0])

(13340, 837)

In [162]:
len(data_source[0].split()), len(data_summary[0].split())

(2412, 152)

In [163]:
def get_sen_encoding(sents):
    '''
    Function to generate encoding for each word in the input list 
    input: sents - List of sentences
    returns the list of the sentence encoding 
    '''
    a = 0.001
    answer = None
    for sent in sents:
        ip =tokenizer(sent, return_tensors='pt', max_length=150, truncation=True, padding='max_length')  # to convert input text into integer ids
        tokens = tokenizer.convert_ids_to_tokens(ip['input_ids'][0])  # converts id into tokens with inclusion of special tokens added during tokenization such as [CLS]
        bert_output = bert_model(**ip)  # BertModel; 
        embedding = bert_output['pooler_output'].clone().detach()
        if answer == None:
            answer = embedding
            answer.resize_(1, 768)
        else:
            embedding.resize_(1, 768)
            answer = torch.cat((answer, embedding),0)
    return answer

In [164]:
from sklearn.metrics.pairwise import cosine_similarity

def similarity_l_l(l1, l2):
    '''
    Function to find the most similar sentence in the document for each sentence in the summary 
    input:  l1 - Summary sentences
            l2 - Document sentences
    returns a list of document sentence indexes for each sentence in the summary 
    '''
    l = l1+l2  # list of sentences in both summary and document
    sents_encodings = get_sen_encoding(l)   # return a list of tensors of length 768 each, for each element of the list
    similarities=cosine_similarity(sents_encodings)   # calculates pairwise similiarities with elements in list if 2 arguments are not passed in; notice along diagonal its almost always equal to 1; https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html#sklearn.metrics.pairwise.cosine_similarity
    
    result = []
    for i in range(len(l1)):
        vals = similarities[i]
        vals = vals[len(l1):]
        idx = np.argmax(vals)
        result.append(idx)
    return result

In [165]:
def nest_sentencesV2(document,chunk_length):
    '''
    function to chunk a document
    input:  document           - Input document
            chunk_length        - chunk length
    output: list of chunks. Each chunk is a list of sentences.
    '''
    nested = []
    sent = []
    length = 0
    for sentence in nltk.sent_tokenize(document):
        length += len(sentence.split(" "))
        if length < chunk_length:
            sent.append(sentence)
        else:
            nested.append(sent)
            sent = []
            sent.append(sentence)
            length = 0
    if len(sent)>0:
        nested.append(sent)
    return nested

In [166]:
def get_chunks_data_from_docV2(doc, summ):
    '''
    Function to generate chunks along with their summaries 
    input:  doc - legal Document
            summ - Gold standard summary
    returns a list of chunks and their summaries 
    '''
    chunk_summ_word_threshold = 75
    sentence_mapping = {}
    doc_sents = split_to_sentences(doc)
    summ_sents = split_to_sentences(summ)
    
    result = (similarity_l_l(summ_sents,doc_sents))
    
    for i in range(len(summ_sents)):
        sentence_mapping[doc_sents[result[i]]] = summ_sents[i]
    
    final_chunks = []
    final_summ = []
    for chunk in nest_sentencesV2(doc, 465):
        summ = ""
        for chunk_sent in chunk:
            if chunk_sent in sentence_mapping:
                summ = summ + sentence_mapping[chunk_sent]
        if len(tokenizer.tokenize(summ)) >= chunk_summ_word_threshold:
            final_chunks.append(" ".join(chunk))
            final_summ.append(summ)
    return final_chunks, final_summ

In [167]:
from tqdm import tqdm
training_chunks = []
training_summs = []
for i in tqdm(range(len(data_source))):
    cks, summs = get_chunks_data_from_docV2(data_source[i],data_summary[i])
    training_chunks = training_chunks + cks
    training_summs = training_summs + summs
#     print(i, len(training_summs), end = ", ", sep = " : ")
    if i%100 == 0: 
        full = list(zip(training_chunks,training_summs))
        df = pd.DataFrame(full,columns=['data', 'summary'])
        df.to_excel("FD_CLS_BK_512.xlsx")
#         break
full = list(zip(training_chunks,training_summs))

100%|██████████| 15/15 [44:29<00:00, 177.95s/it]


In [168]:
df = pd.DataFrame(full,columns=['data', 'summary'])
df.to_excel("FD_CLS_512.xlsx")

##Validation of pre-trained Model

**Have to change path**

In [7]:
filepath = '/content/FD_CLS_512.xlsx'
df = pd.read_excel(filepath,usecols='B,C')

In [8]:
df.shape

(54, 2)

In [9]:
df.head()

,data,summary
0,"Sub sec tion (2) however provided that ""nothin...",The charge in respect of urban immoveable prop...
1,72 of 1950. Petition under article 32 of the C...,"Per PATANJALI SASTRI, MUKHERJEA and DAS JJ.In ..."
2,The petitioner 's case is that the shareholder...,"An incorporated company, therefore, can come u..."
3,"Supposing, howev er, that the impugned Act was...",If on the face of the law there is no classifi...
4,The powers and the duties of the directors are...,The company was made a respondent and opposed ...


In [10]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(df, test_size=0.05,shuffle=False)

In [11]:
train_df.shape,valid_df.shape

((51, 2), (3, 2))

In [12]:
valid_df.reset_index(inplace=True)
valid_df.drop('index',inplace=True,axis=1)

In [13]:
valid_df.head()

,data,summary
0,The view taken by the High Court and which has...,Judgment of the High Court of Madras reversed....
1,If paragraph 5 itself is held to be applicable...,"5,000 to A, the daughter of N."" Later on, amon..."
2,No reliance was placed on section 18 of the Li...,The decree holder appealed contending for the ...


In [14]:
def model_load(checkpoint):
     #model loaded from Huggingface
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
#Model Details 
    print("tokenizer.model_max_length :",tokenizer.model_max_length )# max token inputted in model 1024
    print(" max tokens excluding the special tokens",tokenizer.max_len_single_sentence )# max tokens for input in model 1022
    print("num_special_tokens_to_add ",tokenizer.num_special_tokens_to_add() ) # special token needed by model 2
    return(tokenizer,model)


In [15]:
def sum_generate(FileContent,tokenizer,model):
    inputs = tokenizer(FileContent, return_tensors="pt", max_length=1024, truncation=True) # max_length = 1024 since length exceed otherwise and goes out of index range; genrating inputs with chunks pt stands for pytorch
    output = model.generate(inputs['input_ids'])  # inputs generated many list of tensors, from which 'input_ids' is used for generate  
    summ_generated=tokenizer.decode(output[0], skip_special_tokens=True)# decoding the output  # output[0] used sinced tensor is find in [[]]
    return(summ_generated)

In [16]:
def rouge_score(summaryContent,final_summ_gen):  
    rouge = Rouge()
    result_rouge=rouge.get_scores(summaryContent, final_summ_gen, avg=True)
# Printing the final list
    return result_rouge

In [17]:
# inputs to the model
if __name__ == "__main__":
  checkpoint = "t5-small"
  result_rouge_list = []
  for i in range(valid_df.shape[0]):
    FileContent,summaryContent=valid_df['data'][i], valid_df['summary'][i]
    tokenizer,model=model_load(checkpoint)
    final_summ_gen=sum_generate(FileContent,tokenizer,model)
    result_rouge_list.append(rouge_score(summaryContent,final_summ_gen))

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


tokenizer.model_max_length : 512
 max tokens excluding the special tokens 511
num_special_tokens_to_add  1


/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tokenizer.model_max_length : 512
 max tokens excluding the special tokens 511
num_special_tokens_to_add  1
tokenizer.model_max_length : 512
 max tokens excluding the special tokens 511
num_special_tokens_to_add  1


In [18]:
len(result_rouge_list)

3

In [19]:
agg_rouge_score = {'rouge-1':{'r':0,'p':0,'f':0},'rouge-2':{'r':0,'p':0,'f':0},
                   'rouge-l':{'r':0,'p':0,'f':0}}

for i in range(len(result_rouge_list)):
  for rouge_type in ['rouge-1','rouge-2','rouge-l']:
    for j in ['r','p','f']:
      agg_rouge_score[rouge_type][j]+=(result_rouge_list[i][rouge_type][j]/len(result_rouge_list))

print(agg_rouge_score)

{'rouge-1': {'r': 0.5023809523809524, 'p': 0.04105263157894737, 'f': 0.07580378809495004}, 'rouge-2': {'r': 0.23511904761904762, 'p': 0.012944856226672975, 'f': 0.024313052586233456}, 'rouge-l': {'r': 0.4547619047619048, 'p': 0.03754385964912281, 'f': 0.06926784038253174}}


## Fine-tuning T5 using CLS

In [41]:
train_dataset = Dataset.from_pandas(train_df)                                  
val_dataset = Dataset.from_pandas(valid_df)

In [42]:
train_df.shape

(51, 2)

In [43]:
len(train_df['data'][0].split())

468

In [46]:
datasets = DatasetDict()
datasets['train'] = train_dataset.remove_columns('__index_level_0__')
datasets['valid'] = val_dataset

In [47]:
datasets

DatasetDict({
    train: Dataset({
        features: ['data', 'summary'],
        num_rows: 51
    })
    valid: Dataset({
        features: ['data', 'summary'],
        num_rows: 3
    })
})

In [26]:
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [27]:
max_input_length = 512
max_target_length = 256

def preprocess_function(examples):
  # Reviews are first tokenized
  model_inputs = tokenizer(examples['data'], truncation=True, max_length=max_input_length)
  # tokenizer for the target (highlights)
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples['summary'], truncation=True, max_length=max_target_length)
  # The inputs are first encoded, and then the labels ('highlights') are encoded as a separate column
  model_inputs['labels'] = labels['input_ids']
  return model_inputs

In [28]:
tokenized_dataset = datasets.map(preprocess_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [29]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['data', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 51
    })
    valid: Dataset({
        features: ['data', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3
    })
})

In [30]:
from transformers import AutoModelForSeq2SeqLM

# Seq2SeqLM: sequence-to-sequence language modeling head

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")  # model_checkpoint = "t5-small"

In [38]:
# parameters of Seq2SeqTrainingArguments
batch_size = 8
num_training_epochs = 2
logging_steps = len(tokenized_dataset["train"])//8
model_name = "t5-small-finetuned-cls-part"
model_dir = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/t5-small-finetuned-cls-part.h5"

args = Seq2SeqTrainingArguments(evaluation_strategy='epoch',
                                learning_rate=5.6e-5,
                                per_device_train_batch_size=batch_size,
                                per_device_eval_batch_size=batch_size,
                                weight_decay=0.01,
                                save_total_limit=3,
                                num_train_epochs=num_training_epochs,
                                predict_with_generate=True,
                                logging_steps=logging_steps,
                                output_dir="t5-finetuned-lds-cls-part",
                                push_to_hub=False)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [32]:
def compute_metric(eval_pred):
  predictions, labels = eval_pred
  # Decode generated summaries into text (summaries generated from the review_body)
  # batch_decode -> Returns List[str]; The list of decoded sentences; Convert a list of lists of token ids into a list of strings by calling decode.
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True) # skip_special_tokens - Whether or not to remove special tokens in the decoding.
  # Replace -100 in the labels as we can't decode them
  labels = np.where(labels!=-100, labels, tokenizer.pad_token_id)
  # Decode reference summaries into text ('review_title')
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  # ROUGE expects a newline after each sentence
  # strip() method removes characters from both left and right based on the argument
  decoded_preds = ['\n'.join(sent_tokenize(pred.strip())) for pred in decoded_preds]
  decoded_labels = ['\n'.join(sent_tokenize(pred.strip())) for pred in decoded_labels]
  # Computing ROUGE score
  # Stemming is used to remove plurals and word suffixes such as (ing, ion, ment)
  result = rouge_score.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  result = {key: value*100 for key, value in result.items()}
  return {k: round(v,4) for k, v in result.items()}

In [33]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [34]:
datasets['train'].column_names, len(datasets['train'].column_names)

(['data', 'summary'], 2)

In [35]:
tokenized_dataset = tokenized_dataset.remove_columns(datasets['train'].column_names)
tokenized_dataset['train'].column_names

['input_ids', 'attention_mask', 'labels']

In [36]:
# args = Seq2SeqTrainingArguments()
# model = AutoModelForSeq2SeqLM.from_pretrained('t5-small')
# tokenizer = AutoTokenizer.from_pretrained('t5-small')
trainer = Seq2SeqTrainer(model, args, train_dataset=tokenized_dataset["train"], eval_dataset=tokenized_dataset["valid"], 
                         data_collator=data_collator, tokenizer=tokenizer, compute_metrics=compute_metric)

In [40]:
trainer.train()

***** Running training *****
  Num examples = 51
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 35


Epoch,Training Loss,Validation Loss


***** Running Evaluation *****
  Num examples = 3
  Batch size = 8


Epoch,Training Loss,Validation Loss


AttributeError: ignored

In [ ]:
trainer.evaluate()

**Have to change path**

In [ ]:
model_dir = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/t5-small-finetuned-cls-part.h5"
trainer.save_model(model_dir)

##Validation of fine-tuned Model

In [141]:
def model_load(checkpoint):
     #model loaded from Huggingface
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
#Model Details 
    #print("tokenizer.model_max_length :",tokenizer.model_max_length )# max token inputted in model 1024
    #print(" max tokens excluding the special tokens",tokenizer.max_len_single_sentence )# max tokens for input in model 1022
    #print("num_special_tokens_to_add ",tokenizer.num_special_tokens_to_add() ) # special token needed by model 2
    return(tokenizer,model)


In [142]:
def sum_generate(FileContent,tokenizer,model):
    inputs = tokenizer(FileContent, return_tensors="pt", max_length=1024, truncation=True) # max_length = 1024 since length exceed otherwise and goes out of index range; genrating inputs with chunks pt stands for pytorch
    output = model.generate(inputs['input_ids'])  # inputs generated many list of tensors, from which 'input_ids' is used for generate  
    summ_generated=tokenizer.decode(output[0], skip_special_tokens=True)# decoding the output  # output[0] used sinced tensor is find in [[]]
    return(summ_generated)

In [143]:
def rouge_score(summaryContent,final_summ_gen):  
    rouge = Rouge()
    result_rouge=rouge.get_scores(summaryContent, final_summ_gen, avg=True)
# Printing the final list
    return result_rouge

**Have to change path**

In [144]:
# inputs to the model
if __name__ == "__main__":
  checkpoint = "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/t5-small-finetuned-cls-part.h5"
  result_rouge_list = []
  for i in range(valid_df.shape[0]):
    FileContent,summaryContent=valid_df['data'][i], valid_df['summary'][i]
    tokenizer,model=model_load(checkpoint)
    final_summ_gen=sum_generate(FileContent,tokenizer,model)
    result_rouge_list.append(rouge_score(summaryContent,final_summ_gen))

Didn't find file /content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/t5-small-finetuned-cls-size-130/added_tokens.json. We won't load it.
loading file /content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/t5-small-finetuned-cls-size-130/spiece.model
loading file /content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/t5-small-finetuned-cls-size-130/tokenizer.json
loading file None
loading file /content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/t5-small-finetuned-cls-size-130/special_tokens_map.json
loading file /content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/t5-small-finetuned-cls-size-130/tokenizer_config.json
loading configuration file /content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/t5-small-finetuned-cls-size-130/config.json
Model config T5Config {
  "_name_or_path": "/content/drive/MyDrive/Legal Data/Indian Legal Abs Ext/dataset/IN-Abs/t5-small-finetuned-cls-size-130",
  "

In [145]:
len(result_rouge_list)

15

In [146]:
agg_rouge_score = {'rouge-1':{'r':0,'p':0,'f':0},'rouge-2':{'r':0,'p':0,'f':0},
                   'rouge-l':{'r':0,'p':0,'f':0}}

for i in range(len(result_rouge_list)):
  for rouge_type in ['rouge-1','rouge-2','rouge-l']:
    for j in ['r','p','f']:
      agg_rouge_score[rouge_type][j]+=(result_rouge_list[i][rouge_type][j]/len(result_rouge_list))

print(agg_rouge_score)

{'rouge-1': {'r': 0.33020202020202016, 'p': 0.04780316310364701, 'f': 0.0820922766921572}, 'rouge-2': {'r': 0.08127058127058126, 'p': 0.007673922410429862, 'f': 0.01379944906931314}, 'rouge-l': {'r': 0.31858585858585853, 'p': 0.04555700642501312, 'f': 0.07834985831559248}}
